In [ ]:
from sklearn.multiclass import OutputCodeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
import numpy as np, pandas as pd, os
from prettytable import PrettyTable

In [ ]:
feature_names = list()
for index in range(1, 4):
    feature_names.extend(["Suit"+str(index), "Rank"+str(index)])

feature_names.append('class')


training_input_file = os.path.abspath('poker-hand-training-true.csv')
testing_input_file = os.path.abspath('poker-hand-testing.csv')

np.random.seed(350)    


class myConfigs:
    features = 0
    classes = 0


config = myConfigs()

In [ ]:


train = pd.read_csv(training_input_file, names=feature_names)
test = pd.read_csv(testing_input_file, names=feature_names)


config.features = len(train.columns) - 1
config.classes = len(set(train['class']))


train = train.sample(frac=1).reset_index(drop=True)


trainy = np.array(train['class'])
trainx = np.array(train.drop('class', 1))

testy = np.array(test['class'])
testx = np.array(test.drop('class', 1))


scaler = StandardScaler()

trainset = np.empty(trainx.shape, dtype = float)
testset = np.empty(testx.shape, dtype = float)


for i in range(len(trainx)):
    trainset[i] = trainx[i].astype(float)


for i in range(len(testx)):
    testset[i] = testx[i].astype(float)

scaler.fit(trainset)  


datatrain = scaler.transform(trainset)
datatest = scaler.transform(testset)


In [ ]:
occ = OutputCodeClassifier(BaggingClassifier())
occ.fit(datatrain, trainy)
pred = occ.predict(datatest)
accuracy = accuracy_score(testy, pred)

In [ ]:
print("Accuracy :", round(accuracy * 100, 3))
plt.plot(testy, 'bD', label = "Original")
plt.plot(pred, 'yD', label = "Predicted")
plt.legend()
plt.show()


In [ ]:

conf = [[0] * config.classes for x in range(config.classes)]

for i in range(len(pred)):
    pred1 = pred[i]
    act1 = testy[i]
    conf[act1][pred1] += 1

header = [""]
for l in range(config.classes):
    header.append("Pred " + str(l))
    
title = PrettyTable(header)
l = 0
for c in conf:
    title.add_row(["Real " + str(l)] + c)
    l += 1

print(title)